In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(r"preprocessed_2.csv")

In [3]:
exclude = ['נשירה מחקרית',
 'נשירה קלינית- לאחר ת. טיפול',
 'אין שת"פ טיפולי',
 'סיימו טיפול',
 'אי הסכמה למחקר',
 'אי התאמה למחקר',
 'פספוסי גיוסים']

In [4]:
N = {
    'isin':['CAU', 'IPC-SSC', 'אי הסכמה למחקר', 'אי התאמה למחקר', 'אין שת"פ טיפולי', 'משתתפים פעילים', 'נשירה מחקרית', 'נשירה קלינית- לאחר ת. טיפול', 'סיימו טיפול', 'עלייה לרמה 2', 'פספוסי גיוסים'],
    'not_in': []
}



Eligible = {
            'isin':['CAU', 'IPC-SSC', 'אי הסכמה למחקר', 'אין שת"פ טיפולי', 'משתתפים פעילים', 'נשירה מחקרית', 'נשירה קלינית- לאחר ת. טיפול', 'סיימו טיפול', 'עלייה לרמה 2'],
            'not_in': []
}



Randomized = {
            'isin':['CAU', 'IPC-SSC', 'אין שת"פ טיפולי', 'משתתפים פעילים', 'נשירה מחקרית', 'נשירה קלינית- לאחר ת. טיפול', 'סיימו טיפול', 'עלייה לרמה 2'],
           'not_in': []
}



Dropout = {
           'isin':['נשירה מחקרית', 'נשירה קלינית- לאחר ת. טיפול'],
            'not_in': []
                    }

research_dropout = {
                    'isin':['נשירה מחקרית'],
                    'not_in': []
                    }

clinical_dropout = {
                    'isin':['נשירה קלינית- לאחר ת. טיפול'],
                    'not_in': []
                    }


in_waiting_list = {
    'isin': ['משתתפים פעילים'],
    'not_in': ['CAU', 'IPC-SSC'] + exclude
}


Finished = {
    'isin': ['סיימו טיפול'],
    'not_in': []
}


Active = {
    'isin': ['משתתפים פעילים', 'CAU', 'IPC-SSC'],
    'not_in': exclude
}

not_cooperative = {
    'isin': ['אין שת"פ טיפולי'],
    'not_in': []
}

In [5]:
consort_groups = {
    
    'N': N,
    'Eligible': Eligible,
    'Randomized': Randomized,
    'Dropout': Dropout,
    'Research Dropout': research_dropout,
    'Clinical Dropout': clinical_dropout,
    'In Waiting List': in_waiting_list,
    'Finished': Finished,
    'Active': Active,
    'Not Cooperative': not_cooperative

}

In [6]:


for key, val in consort_groups.items():
    for f in val:
        consort_groups[key][f] = [i for i in consort_groups[key][f] if i not in ['CAU', 'IPC-SSC']]

In [7]:
# Extracting the detials of how to calc the groups 

# rows = []
# for i,j in consort_groups.items():
#     row = {
#         "consort_group": i,
#         "included_tables": j["isin"],
#         "excluded_tables": j["not_in"]
#     }
#     rows.append(row)

# pd.DataFrame(rows).to_csv("consort_groups_definition.csv", index=0)

In [8]:
def isin_consort_group(row, consort_group):
    in_pos_sheet = False
    for sheet in consort_group['isin']:
        if row[sheet]:
            in_pos_sheet = True

    in_neg_sheet = False
    for sheet in consort_group['not_in']:
        if row[sheet]:
            in_neg_sheet = True

    return (in_pos_sheet and (not in_neg_sheet))

In [9]:
for consort_calc in consort_groups.keys():
    print(consort_calc)
    df[consort_calc] = df.apply(isin_consort_group, axis=1, args = [consort_groups[consort_calc]])

N
Eligible
Randomized
Dropout
Research Dropout
Clinical Dropout
In Waiting List
Finished
Active
Not Cooperative


In [10]:
for i in consort_groups.keys():
    print(i, df[i].sum())

N 337
Eligible 197
Randomized 159
Dropout 52
Research Dropout 32
Clinical Dropout 20
In Waiting List 37
Finished 50
Active 37
Not Cooperative 19


In [11]:
start = "first_contact_date"
end = 'therapy_starting_date'

In [12]:
# df[start] = df[start].replace({'0': pd.NaT})
# df[end] = df[end].replace({'0': pd.NaT})


In [13]:
df['waiting_duration'] =  pd.to_datetime(df[end]) - pd.to_datetime(df[start])
df['waiting_duration'] = df['waiting_duration'].dt.days

In [14]:
# filtering out patients that've waited more than an year
df = df[~(df['waiting_duration'] > 670)]

In [15]:
df['did_started_therapy'] = pd.notna(df[end])

In [16]:
df["first_contact_date"]

0      2024-09-22
1      2024-10-08
2      2024-11-14
3      2023-11-23
4      2025-02-13
          ...    
332    2024-11-05
333    2025-04-29
334    2025-03-27
335    2024-12-15
336    2025-02-24
Name: first_contact_date, Length: 336, dtype: object

In [23]:
df.suiteable_for_pp.unique()

array(['1', '0', 'ללא אובדנות', 'לא- הומלץ רק שאלונים', nan, 'רק שאלונים',
       'מעקבים', 'ל.ר. ', 'ל.ר.', 'ל.ר'], dtype=object)

In [22]:
{i: int(i) for i in df.suiteable_for_pp.unique()}

ValueError: invalid literal for int() with base 10: 'ללא אובדנות'

In [ ]:
ל.ר. = nan
ללא אובדנות = nan
לא- הומלץ רק שאלונים = false
רק שאלונים = 

In [18]:
df.groupby('group').suiteable_for_pp.mean()

TypeError: agg function failed [how->mean,dtype->object]

In [17]:
# df.to_csv(r"data\export_with_days_delta.csv", index=0)
df.to_excel(r"data\export_with_days_delta.xlsx", index=0)

In [18]:
df.suiteable_for_pp == '1'

0      False
1       True
2       True
3       True
4       True
       ...  
305     True
306     True
307    False
308    False
309     True
Name: suiteable_for_pp, Length: 309, dtype: bool